In [1]:
# 主要步驟
#     讀取資料
#         讀檔案
#         取出常用變數(若有更改常用變數時要記得更新
#     清洗資料
#         刪除不會用到的欄位
#         刪除不會用到的列
#         找出資料誤值或空值
#         驗算資料
#     轉換資料
#         修正日期格式
#         以代號取代原資料(市場>>代號、花名重編、分類)
#         改成以時間為primary key 的大表格 (full_.csv)
#         以每年group by count 找出 前30名花種取交集刪減資料(reduce_.csv)
#         將上中下價分開(flat_.csv)
#     存取資料

In [ ]:
# 70 64 76 257 83

In [3]:
# seting up 
import os, glob
import numpy as np
import pandas as pd
fn_list = glob.glob("./data_f_price/*.csv")
# print(fn_list)
fn = fn_list[4]

In [4]:
# 讀取檔案 先檢視檔案格式 將常用的變數先列出
df = pd.read_csv(fn, encoding="utf-8")
fn = fn.split("\\")[-1]
print(fn)
df
# 總行數
nraw = df.shape[0]
# 個別欄位名稱
c = df.columns.tolist()
# print(nraw)
# for i in range(len(c)):
#     print(i, c[i])

allRose.csv


In [5]:
# df = pd.read_csv(fn, encoding="utf-8")
# 刪除不使用的資料 
df = df.drop(["增減%","增減%.1","殘貨量"], axis=1)
# 總行數 update
nraw = df.shape[0]
# 個別欄位名稱 update
c = df.columns.tolist()
# c

In [6]:
# 檢視資料內容 類別變相次數分配(順便檢查有沒有key錯的資料)
for colum in ['市場', '產品']:
    freq = df[colum].value_counts()
    dist_value = df[colum].drop_duplicates()
    null_data = df.iloc[df[colum].index.isnull()]
#     print(colum)
#     print(freq)
#     print(dist_value)
#     print("====")
#     print(null_data)
#     print("=====")

In [7]:
flo_name = np.array(df["產品"])
# 將要刪除的index 放進去
ind_to_delete=[]
# n = 10
n = len(flo_name)
for i in range(n):
    data = str(flo_name[i]).split(" ")
#     print(data)
    if len(data)<3:
        ind_to_delete.append(i)
#         print(i, data)
# 刪除有問題的資料 (是小計的那行
df = df.drop(ind_to_delete,axis=0)
df.reset_index(drop=True)
nraw = df.shape[0]

In [8]:
# # 使用 np array 修改資料 檢查資料是否有不一致  FU152 洋桔梗 單瓣綠 FU152有前後不一致名詞
# flo_name = np.array(df["產品"])
# err_code = []
# err_cn =[]
# # n = 100
# dict_flo = {}
# n = len(flo_name)
# for i in range(n):
#     (f_code, f_n, f_cn) = str(flo_name[i]).split(" ")
# #     print(f_code, f_n, f_cn)
#     if f_code in dict_flo:
# #         已經登入的f_code 中有不同的f_cn
#         if f_cn!= dict_flo[f_code]:
#             err_code.append((f_code, dict_flo[f_code], f_cn))
#             err_cn.append(f_cn)
#             print(f_cn)
#     else:
#         dict_flo[f_code] = f_cn
# 檢查沒問題

In [9]:
# 改變時間格式
# 103/01/01 > 2014-01-01
date_old = np.array(df["日期"])
date_new = []
dates = len(date_old)
# dates = 10
for i in range(dates):
    y,m,d = date_old[i].split("/")
    n_date = str(int(y)+1911) + "-" + m + "-" + d
    date_new.append(n_date)
date_time_new = np.asarray(date_new)
df["日期"] = date_time_new

In [10]:
from datetime import datetime, date, time
from datetime import timedelta
d = np.array(df["日期"])
# d

In [11]:
drop_class=[]
for i in range(len(freq)):
    count = freq[i]
    item = freq.index[i]
    if count<500:
        drop_class.append(item)
#         print(count, item)
drop_by_all = drop_class

In [12]:
# d0 = d[0]
# print(d0)
# print(datetime.strptime(d0, "%Y-%m-%d").date())
year = []
dates = len(d)
# dates = 10
for i in range(dates):
#     print(d[i])
    date_data = d[i].split("-")
#     print(date_data[0])
    year.append(date_data[0])
df["year"] = year

In [13]:
# df

In [14]:
# 篩選 產品
# 條件
#     產品總計出現量 > 300
#     每年出現量 > 30

In [15]:
gdf  = df.groupby("year")
year = ["2014", "2015", "2016", "2017", "2018"]
df_by_year=[]
year_rank_30=[]
for y in year:
    t_df = gdf.get_group(y)
    freq = t_df["產品"].value_counts()
    dist_value = df[colum].drop_duplicates()
    # print(freq)
    # print(dist_value)
    r30 = set()
    for i in range(30):
        count = freq[i]
        item = freq.index[i]
#         print(i,count, item)
        r30.add(item)
    year_rank_30.append(r30)


In [16]:
# year_rank_30[0]

In [17]:
for i in range(len(year_rank_30)):
    if i==0:
        all_rank30 = year_rank_30[i]
    all_rank30 = all_rank30.intersection(year_rank_30[i])
# print(len(all_rank30),list(all_rank30))
finall_class = list(all_rank30)

In [18]:
# finall_class

In [19]:
df = df.drop(["year"],axis=1)
# df

In [20]:
fn_full = "full_" + fn
df.to_csv(fn_full, index=False, encoding="utf-8" )

In [21]:
df_arr = np.array(df)
df_arr

array([['2014-01-01', '105 台北花市', 'FR000 玫瑰 ', ..., 111.0, 202.0, 139],
       ['2014-01-01', '400 台中市場', 'FR000 玫瑰 ', ..., 144.0, 150.0, 12],
       ['2014-01-01', '514 彰化市場', 'FR000 玫瑰 ', ..., 99.0, 177.0, 68],
       ...,
       ['2018-12-31', '105 台北花市', 'FR910 玫瑰 迷你紅', ..., 44.0, 67.5, 71],
       ['2018-12-31', '105 台北花市', 'FR930 玫瑰 迷你黃', ..., 85.0, 115.1, 17],
       ['2018-12-31', '105 台北花市', 'FR940 玫瑰 迷你粉', ..., 260.0, 279.1, 11]],
      dtype=object)

In [22]:
n = len(df_arr)
index_in_final=[]
# n = 50
for i in range(n):
    if df_arr[i,2] in finall_class:
#         print(i, df_arr[i,2])
        index_in_final.append(i)
len(index_in_final)
df_arr = df_arr[index_in_final,:]
df_new = pd.DataFrame(df_arr, columns=c)

In [23]:
fn_new = "reduce_"+fn
df_new.to_csv(fn_new, index=False, encoding="utf-8" )
# df_new

In [24]:
test_df = df_new.groupby("日期").get_group("2014-01-01")
test_arr = np.array(test_df)

In [25]:
# test_df

In [26]:
c_new = ["日期", "市場", "產品", "價別", "價格", "交易量"]
# 1 row to 3 new row 
flat_data =[]
for i in range(len(df_arr)):
    o_d = df_arr[i,:]
#     print(o_d)
    l = list(o_d[0:3])
#     print(l)
    p_h = l + ["上價"] +  [int(o_d[4])] + [round(o_d[8]*0.2)]
    p_m = l + ["中價"] +  [int(o_d[5])] + [round(o_d[8]*0.4)]
    p_l = l + ["下價"] +  [int(o_d[6])] + [round(o_d[8]*0.2)]
    flat_data.append(p_h)
    flat_data.append(p_m)
    flat_data.append(p_l)
# flat_data

In [27]:
df_flat = pd.DataFrame(flat_data, columns=c_new)
# df_flat

In [28]:
fn_flat = "flat_"+fn
df_flat.to_csv(fn_flat, index=False, encoding="utf-8" )